Importation données

In [7]:
import pandas as pd
ct = 'QueryResults (1).csv'
df = pd.read_csv(ct)
df

,Id,Title,Body,Tags,Score,ViewCount,AnswerCount
0,20485075,How do I define multiple conditions on a join ...,"<p>Ok, lets say I have order and items tables....",<java><jpa><playframework><playframework-2.0><...,1,1984,2
1,20485079,What is the equivalent of a C# gridview in jav...,"<p>I am trying to learn java, I am working on ...",<c#><java><asp.net><swing><gridview>,2,12333,1
2,20485143,"Listview cursor adapter, always get first item...",<p>I have this piece of code for listview usin...,<android><listview><android-listview><simplecu...,1,1075,2
3,20485144,OpenSSL RSA-2048 unencrypted block is longer t...,<p>I am using the OpenSSL library in order to ...,<c><encryption><cryptography><openssl><rsa>,0,527,1
4,20485176,Returning char into main method,<p>The goal of my program is kind of like whee...,<java><eclipse><string><methods><char>,0,1278,3
...,...,...,...,...,...,...,...
49995,22814994,StructureMap use specific instance of type per...,"<p>Is there a way, using StructureMap (the Dep...",<c#><asp.net><dependency-injection><inversion-...,0,369,2
49996,22815009,Add a reference column migration in Rails 4,<p>A user has many uploads. I want to add a co...,<ruby-on-rails><ruby-on-rails-4><foreign-keys>...,368,397557,8
49997,22815010,Hibernate mapping with group by clause?,<p>I have <code>@OneToMany</code> mapping in a...,<java><sql><hibernate><group-by><hibernate-map...,0,1755,1
49998,22815022,java.lang.NoClassDefFoundError: android.suppor...,<p>I have a class that extends <code>android.s...,<java><android><eclipse><android-fragments><an...,0,1441,1


Filtre des tags

In [8]:
df['Tags'] = df['Tags'].str.replace('><', ',').str.replace('<', '').str.replace('>', '')

from collections import Counter
# Fréquence des mots dans 'Tags'
all_tags = df['Tags'].str.split(',').explode().tolist()
tag_counts = Counter(all_tags)

# Trouver les 50 mots les plus fréquents
top_50_tags = [tag for tag, _ in tag_counts.most_common(50)]

# Filtrer les listes de mots pour ne contenir que les 50 mots les plus fréquents
def filter_tags(tags):
    return [tag for tag in tags if tag in top_50_tags]

df['Tags'] = df['Tags'].str.split(',').apply(filter_tags)

# Filtrer les lignes où 'Tags' est vide après le filtrage
df = df[df['Tags'].apply(len) > 0]

Nettoyage du texte

In [9]:
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, PorterStemmer

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

from bs4 import BeautifulSoup

# Nettoyage
def strip_html_bs(text):
    """nettoyage des balises html"""
    soup = BeautifulSoup(text, 'html.parser')
    return soup.get_text()

def tokenizer_fct(sentence):
    """Division de mots en texte + suppression de certains caractères"""
    sentence_clean = sentence.replace('-', ' ').replace('+', ' ').replace('/', ' ').replace('#', ' ')
    word_tokens = word_tokenize(sentence_clean)
    return word_tokens

stop_w = list(set(stopwords.words('english'))) + ['[', ']', ',', '.', ':', '?', '(', ')','<','>','~']

def stop_word_filter_fct(list_words):
    """Suppression de mots sans information+ ponctuations"""
    filtered_w = [w for w in list_words if not w in stop_w]
    filtered_w2 = [w for w in filtered_w if len(w) > 2]
    return filtered_w2

def lower_start_fct(list_words):
    """Conversion en lettres minuscules et suppression de préfixes indésirables"""
    lw = [w.lower() for w in list_words if (not w.startswith("@")) and (not w.startswith("#")) and (not w.startswith("http"))]
    return lw

def lemma_fct(list_words):
    """lemmatisation"""
    lemmatizer = WordNetLemmatizer()
    lem_w = [lemmatizer.lemmatize(w) for w in list_words]
    return lem_w

def transform_bow_fct(desc_text):
    """fonction de transformation"""
    text_stripped = strip_html_bs(desc_text)
    word_tokens = tokenizer_fct(text_stripped)
    sw = stop_word_filter_fct(word_tokens)
    lw = lower_start_fct(sw)
    lem = lemma_fct(lw) 
    transf_desc_text = ' '.join(lem)
    return transf_desc_text

    
# Prétraitement
df['Cleaned_Body'] = df['Body'].apply(transform_bow_fct)


[nltk_data] Downloading package punkt to
[nltk_data]     /Users/orphila_adjovi/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/orphila_adjovi/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/orphila_adjovi/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
/var/folders/_l/cjjst_c50lx2nkkr280zbp440000gp/T/ipykernel_44290/2665542473.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Cleaned_Body'] = df['Body'].apply(transform_bow_fct)


In [10]:
df['Body'][100]

'<p>I want to grab an image every 3 seconds from an ip camera and put it in a Image control (WPF application).</p>\n\n<p>I\'m using this code:</p>\n\n<p>DispatcherTimer definition</p>\n\n<pre><code>DispatcherTimer dispatcherTimer = new DispatcherTimer(TimeSpan.FromSeconds(3), DispatcherPriority.Background, DispatcherTimer_Tick, Application.Current.Dispatcher);\n</code></pre>\n\n<p>Image grabbing code</p>\n\n<pre><code>private void DispatcherTimer_Tick(object sender, EventArgs evA)\n    {\n        Dispatcher.Invoke(DispatcherPriority.Background,\n         new Action(() =&gt;\n         {\n             try\n             {\n                 BitmapFrame src;\n                 var webRequest = (HttpWebRequest)WebRequest.Create(@"http://camera_ip/cgi-bin/video.cgi?msubmenu=jpg");\n                 webRequest.Credentials = new NetworkCredential("user", "password");\n                 webRequest.Proxy = null;\n\n                 var response = (HttpWebResponse)webRequest.GetResponse();\n        

In [11]:
df['Cleaned_Body'][100]

'want grab image every second camera put image control wpf application using code dispatchertimer definition dispatchertimer dispatchertimer new dispatchertimer timespan.fromseconds dispatcherpriority.background dispatchertimer_tick application.current.dispatcher image grabbing code private void dispatchertimer_tick object sender eventargs eva dispatcher.invoke dispatcherpriority.background new action try bitmapframe src var webrequest httpwebrequest webrequest.create camera_ip cgi bin video.cgi msubmenu=jpg webrequest.credentials new networkcredential user password webrequest.proxy null var response httpwebresponse webrequest.getresponse var stream response.getresponsestream var streamreader new streamreader stream src bitmapframe.create streamreader.basestream imagetelecamera.begininit imagetelecamera.source src imagetelecamera.endinit catch exception dologd error ex.message work fine grab image freeze millisecond why dispatcher working background without affecting'

Comparaison stemming vs lemming

In [17]:
from nltk.stem import PorterStemmer

def stemmer_fct(list_words):
    """Stemming"""
    stemmer = PorterStemmer()
    stemmed_words = [stemmer.stem(w) for w in list_words]
    return stemmed_words
words = ["running", "better", "flies", "happily", "playing", "swimming", "happiest", "jumps", "quickly", "boxes"]

lemmatized_words = lemma_fct(words)
stemmed_words = stemmer_fct(words)

print(f"Words     : {words}")
print(f"Lemmatized: {lemmatized_words}")
print(f"Stemmed   : {stemmed_words}")


Words     : ['running', 'better', 'flies', 'happily', 'playing', 'swimming', 'happiest', 'jumps', 'quickly', 'boxes']
Lemmatized: ['running', 'better', 'fly', 'happily', 'playing', 'swimming', 'happiest', 'jump', 'quickly', 'box']
Stemmed   : ['run', 'better', 'fli', 'happili', 'play', 'swim', 'happiest', 'jump', 'quickli', 'box']
